In [1]:
import numpy as np
import time
import os
import cv2
import kmodel
from utils import transparentOverlay

Using TensorFlow backend.


In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Load Pretrained model
my_model = kmodel.load_trained_model('my_model')

# load face detector
face_cascade = cv2.CascadeClassifier('cascades/haarcascade_frontalface_default.xml')

# Load camera
camera = cv2.VideoCapture(0)

# load sunglasses image
sunglasses = cv2.imread('sunglass.png', cv2.IMREAD_UNCHANGED)

In [3]:
while True:
    time.sleep(0.1)

    # get a picture from camera
    (_, frame) = camera.read()
    frame = cv2.flip(frame, 1)
    frame2 = np.copy(frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #locate all faces (it seems that it can only find one face in each picture)
    faces = face_cascade.detectMultiScale(gray, 1.25, 6)

    # For each face
    for (x, y, w, h) in faces:

        # catch the face part
        gray_face = gray[y:y+h, x:x+w]
        color_face = frame[y:y+h, x:x+w]

        # normalize 
        gray_normalized = gray_face / 255

        # resize the face picture into 96x96 format to fit the input of the NN
        original_shape = gray_face.shape # A Copy for future reference
        face_resized = cv2.resize(gray_normalized, (96, 96), interpolation = cv2.INTER_AREA)
        face_resized = face_resized.reshape(1, 96, 96, 1)

        # predict the coordinate of keypoints
        keypoints = my_model.predict(face_resized)

        # transform the coordinate from (0 to 1) to (0 to 96)
        keypoints = keypoints * 48 + 48

        # resize face_color into 96x96 to match keypoint
        face_resized_color = cv2.resize(color_face, (96, 96), interpolation = cv2.INTER_AREA)
        face_resized_color2 = np.copy(face_resized_color)

        # put the 30 output digits from the NN into 15 turples
        points = []
        for i, co in enumerate(keypoints[0][0::2]):
            points.append((co, keypoints[0][1::2][i]))

        # find the width of sunglass left_eyebrow_outer_end_x[7], right_eyebrow_outer_end_x[9]
        sunglass_width = int((points[7][0]-points[9][0])*1.1)

        # find the height of sunglass nose_tip_y[10], right_eyebrow_inner_end_y[8]
        sunglass_height = int((points[10][1]-points[8][1])/1.1)
        sunglass_resized = cv2.resize(sunglasses, (sunglass_width, sunglass_height), interpolation = cv2.INTER_CUBIC)
        face_resized_color = transparentOverlay(face_resized_color, sunglass_resized , pos=(int(points[9][0]),int(points[9][1])), scale = 1)

        # resize face_resized_color(with sunglasses) to original picture
        frame[y:y+h, x:x+w] = cv2.resize(face_resized_color, original_shape, interpolation = cv2.INTER_CUBIC)

        # show the keypoints in face picture
        for keypoint in points:
            cv2.circle(face_resized_color2, keypoint, 1, (0,255,0), 1)

        frame2[y:y+h, x:x+w] = cv2.resize(face_resized_color2, original_shape, interpolation = cv2.INTER_CUBIC)

        # show the picture with glass
        cv2.imshow("With Glass", frame)
        # show the picture with keypoints
        cv2.imshow("With Keypoints", frame2)

    # when we click q, clost the picture
    #if keyboard.is_pressed('q'):
    #    break
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

KeyboardInterrupt: 

In [4]:
# release camera, close the window
camera.release()
cv2.destroyAllWindows()